In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
#1. 필요한 라이브러리 불러오기
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report, accuracy_score, f1_score
import lightgbm as lgb

#2. 데이터 로드
data = pd.read_csv("/content/drive/MyDrive/monunmon.csv")  # CSV 파일 경로

#3. 레이블 이진 분류를 위해 변환 (0~94 -> 1, -1 -> 0)
data['Label'] = data['Label'].apply(lambda x: 1 if x >= 0 else 0)

#4. 변환 결과 확인
print("Label Distribution:")
print(data['Label'].value_counts())

#5. 특성과 레이블 분리
X = data.drop(columns=['Label'])  # 피처 데이터
y = data['Label']  # 레이블 데이터

#6. Train, Validation, Test 데이터 나누기
X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.3, random_state=42, stratify=y)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42, stratify=y_temp)

#7. 데이터 정규화 (LightGBM은 정규화 필요 없음, 하지만 데이터 분포에 따라 적용)
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_val = scaler.transform(X_val)
X_test = scaler.transform(X_test)

#8. LightGBM 모델 정의 및 학습
model = lgb.LGBMClassifier(random_state=42, n_estimators=1000, learning_rate=0.1, max_depth=-1)
model.fit(X_train, y_train)

#9. 검증 데이터 평가
y_val_pred = model.predict(X_val)
val_accuracy = accuracy_score(y_val, y_val_pred)
val_f1 = f1_score(y_val, y_val_pred)

print(f"Validation Accuracy: {val_accuracy:.2f}")
print(f"Validation F1 Score: {val_f1:.2f}")

#10. 테스트 데이터 평가
y_test_pred = model.predict(X_test)
test_accuracy = accuracy_score(y_test, y_test_pred)
test_f1 = f1_score(y_test, y_test_pred)

print(f"Test Accuracy: {test_accuracy:.2f}")
print(f"Test F1 Score: {test_f1:.2f}")

#11. 분류 보고서 출력
print("\nClassification Report on Test Data:")
print(classification_report(y_test, y_test_pred))

Label Distribution:
Label
1    19000
0     3000
Name: count, dtype: int64
[LightGBM] [Info] Number of positive: 13300, number of negative: 2100
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.003941 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 3345
[LightGBM] [Info] Number of data points in the train set: 15400, number of used features: 15
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.863636 -> initscore=1.845827
[LightGBM] [Info] Start training from score 1.845827
Validation Accuracy: 0.92
Validation F1 Score: 0.95
Test Accuracy: 0.91
Test F1 Score: 0.95

Classification Report on Test Data:
              precision    recall  f1-score   support

           0       0.80      0.45      0.58       450
           1       0.92      0.98      0.95      2850

    accuracy                           0.91      3300
   macro avg       0.86      0.72      0.76      3300
weighted avg       0.90      0.91 

In [ ]:
#1. 필요한 라이브러리 불러오기
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report, accuracy_score, f1_score
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import RandomUnderSampler
import lightgbm as lgb

#2. 데이터 로드
data = pd.read_csv("/content/drive/MyDrive/monunmon.csv")  # 데이터셋 경로 입력
data['Label'] = data['Label'].apply(lambda x: 1 if x >= 0 else 0)  # 이진 레이블 변환

#3. 특성과 레이블 분리
X = data.drop(columns=['Label'])  # 피처 데이터
y = data['Label']  # 레이블 데이터

#4. Train, Validation, Test 데이터 나누기
X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.3, random_state=42, stratify=y)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42, stratify=y_temp)

#5. 데이터 정규화 (LightGBM은 정규화 불필요하지만 형식 유지)
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_val = scaler.transform(X_val)
X_test = scaler.transform(X_test)

### 방법 1: 클래스 가중치 적용 ###
print("=== LightGBM with Class Weights ===")
model_weighted = lgb.LGBMClassifier(random_state=42, class_weight='balanced', n_estimators=1000, learning_rate=0.1)
model_weighted.fit(X_train, y_train)

# 검증 데이터 평가
y_val_pred_weighted = model_weighted.predict(X_val)
val_accuracy_weighted = accuracy_score(y_val, y_val_pred_weighted)
val_f1_weighted = f1_score(y_val, y_val_pred_weighted)
print(f"Validation Accuracy: {val_accuracy_weighted:.2f}")
print(f"Validation F1 Score: {val_f1_weighted:.2f}")

# 테스트 데이터 평가
y_test_pred_weighted = model_weighted.predict(X_test)
test_accuracy_weighted = accuracy_score(y_test, y_test_pred_weighted)
test_f1_weighted = f1_score(y_test, y_test_pred_weighted)
print(f"Test Accuracy: {test_accuracy_weighted:.2f}")
print(f"Test F1 Score: {test_f1_weighted:.2f}")

# 분류 보고서
print("\nClassification Report on Test Data (Class Weights):")
print(classification_report(y_test, y_test_pred_weighted))

### 방법 2: SMOTE를 이용한 오버샘플링 ###
print("\n=== LightGBM with SMOTE ===")
smote = SMOTE(random_state=42)
X_train_smote, y_train_smote = smote.fit_resample(X_train, y_train)

print("Resampled Training Set Size (SMOTE):", X_train_smote.shape)

model_smote = lgb.LGBMClassifier(random_state=42, n_estimators=1000, learning_rate=0.1)
model_smote.fit(X_train_smote, y_train_smote)

# 검증 데이터 평가
y_val_pred_smote = model_smote.predict(X_val)
val_accuracy_smote = accuracy_score(y_val, y_val_pred_smote)
val_f1_smote = f1_score(y_val, y_val_pred_smote)
print(f"Validation Accuracy: {val_accuracy_smote:.2f}")
print(f"Validation F1 Score: {val_f1_smote:.2f}")

# 테스트 데이터 평가
y_test_pred_smote = model_smote.predict(X_test)
test_accuracy_smote = accuracy_score(y_test, y_test_pred_smote)
test_f1_smote = f1_score(y_test, y_test_pred_smote)
print(f"Test Accuracy: {test_accuracy_smote:.2f}")
print(f"Test F1 Score: {test_f1_smote:.2f}")

# 분류 보고서
print("\nClassification Report on Test Data (SMOTE):")
print(classification_report(y_test, y_test_pred_smote))

### 방법 3: 언더샘플링 ###
print("\n=== LightGBM with Undersampling ===")
undersampler = RandomUnderSampler(random_state=42)
X_train_under, y_train_under = undersampler.fit_resample(X_train, y_train)

print("Resampled Training Set Size (Undersampling):", X_train_under.shape)

model_under = lgb.LGBMClassifier(random_state=42, n_estimators=1000, learning_rate=0.1)
model_under.fit(X_train_under, y_train_under)

# 검증 데이터 평가
y_val_pred_under = model_under.predict(X_val)
val_accuracy_under = accuracy_score(y_val, y_val_pred_under)
val_f1_under = f1_score(y_val, y_val_pred_under)
print(f"Validation Accuracy: {val_accuracy_under:.2f}")
print(f"Validation F1 Score: {val_f1_under:.2f}")

# 테스트 데이터 평가
y_test_pred_under = model_under.predict(X_test)
test_accuracy_under = accuracy_score(y_test, y_test_pred_under)
test_f1_under = f1_score(y_test, y_test_pred_under)
print(f"Test Accuracy: {test_accuracy_under:.2f}")
print(f"Test F1 Score: {test_f1_under:.2f}")

# 분류 보고서
print("\nClassification Report on Test Data (Undersampling):")
print(classification_report(y_test, y_test_pred_under))


=== LightGBM with Class Weights ===
[LightGBM] [Info] Number of positive: 13300, number of negative: 2100
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.002219 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 3345
[LightGBM] [Info] Number of data points in the train set: 15400, number of used features: 15
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.500000 -> initscore=-0.000000
[LightGBM] [Info] Start training from score -0.000000
Validation Accuracy: 0.91
Validation F1 Score: 0.95
Test Accuracy: 0.91
Test F1 Score: 0.95

Classification Report on Test Data (Class Weights):
              precision    recall  f1-score   support

           0       0.71      0.55      0.62       450
           1       0.93      0.96      0.95      2850

    accuracy                           0.91      3300
   macro avg       0.82      0.76      0.78      3300
weighted avg       0.90      0.91      0.90      3300
